In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd
sys.path.append('/Users/ajc/CODES/ats/tools/utils/')
import get_transect_data

In [3]:
Data_B = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_B_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_B['temperature'] = np.array(Data)
Data_B['time'] = np.array(Time)

In [4]:
Data_S30 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S30_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S30['temperature'] = np.array(Data)
Data_S30['time'] = np.array(Time)

In [5]:
#print Data_B['temperature'].shape
#print Data_B['time'][0], Data_B['time'][-1]
#print Data_EF['temperature'].shape

In [6]:
Data_S60 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S60_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S60['temperature'] = np.array(Data)
Data_S60['time'] = np.array(Time)

In [7]:
Data_S90 = dict()
infile='/Users/ajc/mytests/simulations/hillslope/2019/MaxSnow/MS_S90_1R1/'
Time, Data = get_transect_data.get2DMatrix(infile)
Data_S90['temperature'] = np.array(Data)
Data_S90['time'] = np.array(Time)

In [8]:
"""
    nvar, cycles, xnum, znum = data.shape
    RiparianT = dict()
    avgTemp = np.zeros(cycles)
    for i in range(cycles):
        avgTemp[i] = np.mean(data[nvar-1,i,0:10,55:])
    RiparianT['times'] = np.array(times)
    RiparianT['temp'] = np.array(avgTemp)
    for key in RiparianT.keys():
        RiparianT[key + '-annual_avg'] = np.reshape(RiparianT[key], (-1,365)).mean(axis=1)    
    return RiparianT
"""

"\n    nvar, cycles, xnum, znum = data.shape\n    RiparianT = dict()\n    avgTemp = np.zeros(cycles)\n    for i in range(cycles):\n        avgTemp[i] = np.mean(data[nvar-1,i,0:10,55:])\n    RiparianT['times'] = np.array(times)\n    RiparianT['temp'] = np.array(avgTemp)\n    for key in RiparianT.keys():\n        RiparianT[key + '-annual_avg'] = np.reshape(RiparianT[key], (-1,365)).mean(axis=1)    \n    return RiparianT\n"

In [9]:
def get_AveMaxMin_Temp(season = " ", start=0, end=-1,depth=25):
    #25 * 2cm = 50 cm
    data = []
    times = []
    if season == "B":
        data = Data_B['temperature']
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30['temperature']
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60['temperature']
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90['temperature']
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeT = dict()
    
    AvgTemp = np.zeros(cycles)
    MaxTemp = np.zeros(cycles)
    MinTemp = np.zeros(cycles)
    #print depth
    depth_cm = data[1,0,0,80] - data[1,0,0,80-depth]
    for i in range(cycles):
        AvgTemp[i] = np.mean(data[nvar-1,i,start:end,80-depth:])
        MaxTemp[i] = np.max(data[nvar-1,i,start:end,80-depth:])
        MinTemp[i] = np.min(data[nvar-1,i,start:end,80-depth:])
    L = len(AvgTemp)%365
    HillslopeT['times'] = np.array(times[:-L])
    HillslopeT_Avg = np.array(AvgTemp[:-L])
    HillslopeT_Max = np.array(MaxTemp[:-L])
    HillslopeT_Min = np.array(MinTemp[:-L])
   

    HillslopeT['temp-annual_avg'] = np.reshape(HillslopeT_Avg, (-1,365)).mean(axis=1)
    HillslopeT['temp-annual_max'] = np.reshape(HillslopeT_Max, (-1,365)).max(axis=1)
    HillslopeT['temp-annual_min'] = np.reshape(HillslopeT_Min, (-1,365)).min(axis=1)

    return HillslopeT, depth_cm

In [23]:
def get_locationbased_ALT(season = " ",location=0):
    data = []
    times = []
    if season == "B":
        data = Data_B['temperature']
        times = Data_B['time']
    elif season == "S30":
        data = Data_S30['temperature']
        times = Data_S30['time']
    elif season == "S60":
        data = Data_S60['temperature']
        times = Data_S60['time']
    elif season == "S90":
        data = Data_S90['temperature']
        times = Data_S90['time']
        
    nvar, cycles, xnum, znum = data.shape
    HillslopeT = dict()
    avgTemp = np.zeros(cycles)
    
    ddepth = 0 
    for i in range(cycles):
        loc = np.where(data[nvar-1,i,location,:] > 273.15)[0]
        if len(loc) >0:
            dz = -(80 - loc[0])*2.0
            avgTemp[i] = dz
    HillslopeT['times'] = np.array(times)
    HillslopeT['temp'] = np.array(avgTemp)
    print nvar,len(loc), loc 
    print data[1,0,location,:], data[1,0,location,loc[0]], loc[0]
    L = len(HillslopeT['temp'])%365
    for key in HillslopeT.keys():
        HillslopeT[key + '-annual_avg'] = np.reshape(HillslopeT[key][:-L], (-1,365))   
    return HillslopeT

In [18]:
RiparianTemp = dict()
#start1_,end1_,depth_ = 0,10,55 # Z depth 3.84718
start1_,end1_,depth_ = 0,10,46 # Z depth 2.053

RiparianTempB, Depth_cm = get_AveMaxMin_Temp("B",start=start1_,end=end1_,depth=depth_)
RiparianTempS30,Depth_cm = get_AveMaxMin_Temp("S30",start=start1_,end=end1_,depth=depth_)
RiparianTempS60,Depth_cm = get_AveMaxMin_Temp("S60",start=start1_,end=end1_,depth=depth_)
RiparianTempS90,Depth_cm = get_AveMaxMin_Temp("S90",start=start1_,end=end1_,depth=depth_)
print (Depth_cm)

2.0531099999999998


In [ ]:
depth_cm =round(100*Depth_cm)

In [ ]:
HillslopeTemp = dict()
start2_,end2_ = 11,99
HillslopeTempB,Depth_cm = get_AveMaxMin_Temp("B",start=start2_,end=end2_,depth=depth_)
HillslopeTempS30,Depth_cm = get_AveMaxMin_Temp("S30",start=start2_,end=end2_,depth=depth_)
HillslopeTempS60,Depth_cm = get_AveMaxMin_Temp("S60",start=start2_,end=end2_,depth=depth_)
HillslopeTempS90,Depth_cm = get_AveMaxMin_Temp("S90",start=start2_,end=end2_,depth=depth_)

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAverage():
    fig, axs = plt.subplots(1,2, figsize=(8,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    axs = axs.ravel()
    #plt.subplot(121)
    ax1=plt.subplot(121)
    ax1.plot(RiparianTempB['times-annual_avg'], RiparianTempB['temp-annual_avg'],'k.-',label='Basecase')
    ax1.plot(RiparianTempS30['times-annual_avg'], RiparianTempS30['temp-annual_avg'],'g.-',label=r'S$_{30}$')
    ax1.plot(RiparianTempS60['times-annual_avg'], RiparianTempS60['temp-annual_avg'],'b.-',label=r'S$_{60}$')
    ax1.plot(RiparianTempS90['times-annual_avg'], RiparianTempS90['temp-annual_avg'],'m.-',label=r'S$_{90}$')

    ax1.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Annual average temperature [K]',fontsize='large', fontweight='normal')
    #ax1.set_xlim((25., 31))
    ax1.set_ylim((266.5, 271.5))
    #ax1.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    ax1.legend(loc='lower left',fontsize=8)
    ax1.set_title('Wetland',fontweight='normal')
    plt.tight_layout()

    ax2=plt.subplot(122)
    ax2.plot(HillslopeTempB['times-annual_avg'], HillslopeTempB['temp-annual_avg'],'k.-',label='Basecase')
    ax2.plot(HillslopeTempS30['times-annual_avg'], HillslopeTempS30['temp-annual_avg'],'g.-',label=r'S$_{30}$')
    ax2.plot(HillslopeTempS60['times-annual_avg'], HillslopeTempS60['temp-annual_avg'],'b.-',label=r'S$_{60}$')
    ax2.plot(HillslopeTempS90['times-annual_avg'], HillslopeTempS90['temp-annual_avg'],'m.-',label=r'S$_{90}$')


    ax2.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax2.set_ylabel('Annual average temperature [K]',fontsize='large', fontweight='normal')
    #ax2.set_xlim((25., 31))
    ax2.set_ylim((266.5, 271.5))
    #ax2.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    ax2.legend(loc='upper left',fontsize=8)
    ax2.set_title('Upland',fontweight='normal')

    plt.tight_layout()
    #print 'DONE'
    plt.savefig(outfile+'temp-%scm.pdf'%depth_cm, bbox_inches='tight', dpi=100)
    plt.show()

    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAvgMaxMin():
    fig, axs = plt.subplots(3,2, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    axs = axs.ravel()
    #plt.subplot(121)
    ax1=plt.subplot(321)
    ax1.plot(RiparianTempB['times-annual_avg'], RiparianTempB['temp-annual_avg'],'k.-',label='Basecase')
    ax1.plot(RiparianTempS30['times-annual_avg'], RiparianTempS30['temp-annual_avg'],'g.-',label=r'S$_{30}$')
    ax1.plot(RiparianTempS60['times-annual_avg'], RiparianTempS60['temp-annual_avg'],'b.-',label=r'S$_{60}$')
    ax1.plot(RiparianTempS90['times-annual_avg'], RiparianTempS90['temp-annual_avg'],'m.-',label=r'S$_{90}$')

    #ax1.set_xlabel('Year',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    #ax1.set_xlim((25., 31))
    ax1.set_ylim((266.5, 271.5))
    #ax1.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax1.legend(loc='upper left',fontsize=8)
    ax1.set_title('Wetland',fontweight='normal')
    ax1.text(2, 267, 'Annual average temperature',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(322)
    ax2.plot(HillslopeTempB['times-annual_avg'], HillslopeTempB['temp-annual_avg'],'k.-',label='Basecase')
    ax2.plot(HillslopeTempS30['times-annual_avg'], HillslopeTempS30['temp-annual_avg'],'g.-',label=r'S$_{30}$')
    ax2.plot(HillslopeTempS60['times-annual_avg'], HillslopeTempS60['temp-annual_avg'],'b.-',label=r'S$_{60}$')
    ax2.plot(HillslopeTempS90['times-annual_avg'], HillslopeTempS90['temp-annual_avg'],'m.-',label=r'S$_{90}$')


    #ax2.set_xlabel('Year',fontsize='large', fontweight='normal')
    #ax2.set_ylabel('Annual average temperature [K]',fontsize='large', fontweight='normal')
    #ax2.set_xlim((25., 31))
    ax2.set_ylim((266.5, 271.5))
    #ax2.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax2.legend(loc='upper left',fontsize=8)
    ax2.set_title('Upland',fontweight='normal')
    ax2.text(2, 270.5, 'Annual average temperature',fontsize=11, fontweight='normal')
    
    # --------------------------------- MAXIMUM ---------------------------
    ax1=plt.subplot(323)
    ax1.plot(RiparianTempB['times-annual_max'], RiparianTempB['temp-annual_max'],'k.-',label='Basecase')
    ax1.plot(RiparianTempS30['times-annual_max'], RiparianTempS30['temp-annual_max'],'g.-',label=r'S$_{30}$')
    ax1.plot(RiparianTempS60['times-annual_max'], RiparianTempS60['temp-annual_max'],'b.-',label=r'S$_{60}$')
    ax1.plot(RiparianTempS90['times-annual_max'], RiparianTempS90['temp-annual_max'],'m.-',label=r'S$_{90}$')

    #ax1.set_xlabel('Year',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    #ax1.set_xlim((25., 31))
    ax1.set_ylim((275.5, 276.2))
    #ax1.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax1.set_yticklabels(np.linspace(275.6,276.2,4))
    #ax1.legend(loc='upper left',fontsize=8)
    ax1.text(2, 275.6, 'Annual maximum temperature',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(324)
    ax2.plot(HillslopeTempB['times-annual_max'], HillslopeTempB['temp-annual_max'],'k.-',label='Basecase')
    ax2.plot(HillslopeTempS30['times-annual_max'], HillslopeTempS30['temp-annual_max'],'g.-',label=r'S$_{30}$')
    ax2.plot(HillslopeTempS60['times-annual_max'], HillslopeTempS60['temp-annual_max'],'b.-',label=r'S$_{60}$')
    ax2.plot(HillslopeTempS90['times-annual_max'], HillslopeTempS90['temp-annual_max'],'m.-',label=r'S$_{90}$')

    #ax2.set_xlabel('Year',fontsize='large', fontweight='normal')
    #ax2.set_ylabel('Annual average temperature [K]',fontsize='large', fontweight='normal')
    #ax2.set_xlim((25., 31))
    ax2.set_ylim((275.5, 276.2))
    ax2.text(2, 275.6, 'Annual maximum temperature',fontsize=11, fontweight='normal')
    #ax2.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax2.legend(loc='upper left',fontsize=8)
    
    # --------------------------------- MANIMUM ---------------------------
    ax1=plt.subplot(325)
    ax1.plot(RiparianTempB['times-annual_min'], RiparianTempB['temp-annual_min'],'k.-',label='Basecase')
    ax1.plot(RiparianTempS30['times-annual_min'], RiparianTempS30['temp-annual_min'],'g.-',label=r'S$_{30}$')
    ax1.plot(RiparianTempS60['times-annual_min'], RiparianTempS60['temp-annual_min'],'b.-',label=r'S$_{60}$')
    ax1.plot(RiparianTempS90['times-annual_min'], RiparianTempS90['temp-annual_min'],'m.-',label=r'S$_{90}$')

    ax1.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    #ax1.set_xlim((25., 31))
    ax1.set_ylim((256., 266))
    #ax1.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax1.legend(loc='upper left',fontsize=8)
    ax1.text(2, 257., 'Annual minimum temperature',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(326)
    ax2.plot(HillslopeTempB['times-annual_min'], HillslopeTempB['temp-annual_min'],'k.-',label='Basecase')
    ax2.plot(HillslopeTempS30['times-annual_min'], HillslopeTempS30['temp-annual_min'],'g.-',label=r'S$_{30}$')
    ax2.plot(HillslopeTempS60['times-annual_min'], HillslopeTempS60['temp-annual_min'],'b.-',label=r'S$_{60}$')
    ax2.plot(HillslopeTempS90['times-annual_min'], HillslopeTempS90['temp-annual_min'],'m.-',label=r'S$_{90}$')

    ax2.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    #ax2.set_ylabel('Annual average temperature [K]',fontsize='large', fontweight='normal')
    #ax2.set_xlim((25., 31))
    ax2.set_ylim((256., 266))
    #ax2.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
    #ax2.legend(loc='upper left',fontsize=8)
    ax2.text(2, 264.5, 'Annual minimum temperature',fontsize=11, fontweight='normal')
    ax2.legend(loc='upper right', fontsize=10, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    
    
    plt.tight_layout()
    plt.savefig(outfile+'avg-max-min-temp-%scm.pdf'%depth_cm, bbox_inches='tight', dpi=100)
    plt.show()

    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAvgMaxMinBAR():
    fig, axs = plt.subplots(3,2, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 273.15
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = ['Basecase', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(321)
    X = [RiparianTempB['temp-annual_avg'][-1], RiparianTempS30['temp-annual_avg'][-1],RiparianTempS60['temp-annual_avg'][-1],RiparianTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    ax1.set_ylim((266.5 - TempC, 271.5- TempC))
    ax1.set_title('Wetland',fontweight='normal')
    ax1.text(-0.2, 271- TempC, 'Annual average',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(322)
    X = [HillslopeTempB['temp-annual_avg'][-1], HillslopeTempS30['temp-annual_avg'][-1],HillslopeTempS60['temp-annual_avg'][-1],HillslopeTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    ax2.set_ylim((266.5- TempC, 271.5- TempC))
    ax2.set_title('Upland',fontweight='normal')
    ax2.text(-0.2, 270.5- TempC, 'Annual average',fontsize=11, fontweight='normal')
    
    # --------------------------------- MAXIMUM ---------------------------
    ax1=plt.subplot(323)
    X = [RiparianTempB['temp-annual_max'][-1], RiparianTempS30['temp-annual_max'][-1],RiparianTempS60['temp-annual_max'][-1],RiparianTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    ax1.set_ylim((275.5- TempC, 276.5- TempC))
    ax1.text(-0.2, 276.4- TempC, 'Annual maximum',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(324)
    X = [HillslopeTempB['temp-annual_max'][-1], HillslopeTempS30['temp-annual_max'][-1],HillslopeTempS60['temp-annual_max'][-1],HillslopeTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    ax2.set_ylim((275.5- TempC, 276.5- TempC))
    ax2.text(-0.2, 276.4- TempC, 'Annual maximum',fontsize=11, fontweight='normal')
    
    # --------------------------------- MANIMUM ---------------------------
    ax1=plt.subplot(325)
    X = [RiparianTempB['temp-annual_min'][-1], RiparianTempS30['temp-annual_min'][-1],RiparianTempS60['temp-annual_min'][-1],RiparianTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    #ax1.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    
    ax1.set_ylim((256.- TempC, 266- TempC))
    ax1.text(-0.2, 264.6- TempC, 'Annual minimum',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(326)
    X = [HillslopeTempB['temp-annual_min'][-1], HillslopeTempS30['temp-annual_min'][-1],HillslopeTempS60['temp-annual_min'][-1],HillslopeTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    
    #ax2.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax2.set_ylim((256.- TempC, 266- TempC))
    ax2.text(-0.2, 264.5- TempC, 'Annual minimum',fontsize=11, fontweight='normal')
    ax2.legend(loc='upper right', fontsize=10, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    
    plt.tight_layout()
    plt.savefig(outfile+'avg-max-min-temp-50cm-bar.pdf', bbox_inches='tight', dpi=100)
    plt.show()

    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAvgMaxMinBARSingleColumn():
    fig, axs = plt.subplots(3,1, figsize=(6,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 273.15
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = ['Basecase', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(311)
    X = [RiparianTempB['temp-annual_avg'][-1], RiparianTempS30['temp-annual_avg'][-1],RiparianTempS60['temp-annual_avg'][-1],RiparianTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='black')
    
    X = [HillslopeTempB['temp-annual_avg'][-1], HillslopeTempS30['temp-annual_avg'][-1],HillslopeTempS60['temp-annual_avg'][-1],HillslopeTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos+0.25,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='cyan', hatch="/")
    
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    ax1.set_ylim((266.5 - TempC, 271.5- TempC))
    #ax1.set_title('Wetland',fontweight='normal')
    ax1.text(-0.2, 271- TempC, 'Annual average',fontsize=11, fontweight='normal')
    plt.tight_layout()

    # --------------------------------- MAXIMUM ---------------------------
    ax2=plt.subplot(312)
    X = [RiparianTempB['temp-annual_max'][-1], RiparianTempS30['temp-annual_max'][-1],RiparianTempS60['temp-annual_max'][-1],RiparianTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='black')
    
    X = [HillslopeTempB['temp-annual_max'][-1], HillslopeTempS30['temp-annual_max'][-1],HillslopeTempS60['temp-annual_max'][-1],HillslopeTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos+0.25,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='cyan', hatch="/")
    
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax2.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    
    ax2.set_ylim((275.5- TempC, 276.5- TempC))
    #ax1.set_title('Wetland',fontweight='normal')
    ax2.text(-0.2, 271- TempC, 'Annual average',fontsize=11, fontweight='normal')
    plt.tight_layout()
    
    # --------------------------------- MANIMUM ---------------------------
    ax3=plt.subplot(313)
    X = [RiparianTempB['temp-annual_min'][-1], RiparianTempS30['temp-annual_min'][-1],RiparianTempS60['temp-annual_min'][-1],RiparianTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax3.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='black')
    
    X = [HillslopeTempB['temp-annual_min'][-1], HillslopeTempS30['temp-annual_min'][-1],HillslopeTempS60['temp-annual_min'][-1],HillslopeTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax3.bar(xpos+0.25,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.25,edgecolor='cyan', hatch="/")
    
    ax3.set_xticks(xpos)
    ax3.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax3.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    
    #ax2.set_ylim((275.5- TempC, 276.5- TempC))
    #ax1.set_title('Wetland',fontweight='normal')
    ax3.set_ylim((256.- TempC, 266- TempC))
    ax3.text(-0.2, 271- TempC, 'Annual average',fontsize=11, fontweight='normal')
    plt.tight_layout()
    
    """
    ax2=plt.subplot(312)
    X = [HillslopeTempB['temp-annual_avg'][-1], HillslopeTempS30['temp-annual_avg'][-1],HillslopeTempS60['temp-annual_avg'][-1],HillslopeTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    ax2.set_ylim((266.5- TempC, 271.5- TempC))
    ax2.set_title('Upland',fontweight='normal')
    ax2.text(-0.2, 270.5- TempC, 'Annual average',fontsize=11, fontweight='normal')
    
    # --------------------------------- MAXIMUM ---------------------------
    ax1=plt.subplot(313)
    X = [RiparianTempB['temp-annual_max'][-1], RiparianTempS30['temp-annual_max'][-1],RiparianTempS60['temp-annual_max'][-1],RiparianTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    ax1.set_ylim((275.5- TempC, 276.5- TempC))
    ax1.text(-0.2, 276.4- TempC, 'Annual maximum',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(324)
    X = [HillslopeTempB['temp-annual_max'][-1], HillslopeTempS30['temp-annual_max'][-1],HillslopeTempS60['temp-annual_max'][-1],HillslopeTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    ax2.set_ylim((275.5- TempC, 276.5- TempC))
    ax2.text(-0.2, 276.4- TempC, 'Annual maximum',fontsize=11, fontweight='normal')
    
    # --------------------------------- MANIMUM ---------------------------
    ax1=plt.subplot(325)
    X = [RiparianTempB['temp-annual_min'][-1], RiparianTempS30['temp-annual_min'][-1],RiparianTempS60['temp-annual_min'][-1],RiparianTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax1.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    #ax1.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Soil temperature [K]',fontsize='large', fontweight='normal')
    
    ax1.set_ylim((256.- TempC, 266- TempC))
    ax1.text(-0.2, 264.6- TempC, 'Annual minimum',fontsize=11, fontweight='normal')
    plt.tight_layout()

    
    ax2=plt.subplot(326)
    X = [HillslopeTempB['temp-annual_min'][-1], HillslopeTempS30['temp-annual_min'][-1],HillslopeTempS60['temp-annual_min'][-1],HillslopeTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    ax2.bar(xpos,X,align='center', alpha=1.0, color=['b', 'k','g','r'], width=0.5)
    ax2.set_xticks(xpos)
    ax2.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    
    #ax2.set_xlabel('Time [year]',fontsize='large', fontweight='normal')
    ax2.set_ylim((256.- TempC, 266- TempC))
    ax2.text(-0.2, 264.5- TempC, 'Annual minimum',fontsize=11, fontweight='normal')
    ax2.legend(loc='upper right', fontsize=10, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    
    plt.tight_layout()
    plt.savefig(outfile+'avg-max-min-temp-50cm-bar.pdf', bbox_inches='tight', dpi=100)
    plt.show()
    """
    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [ ]:
PlotAnnualAvgMaxMinBARSingleColumn()

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
def PlotAnnualAvgMaxMinLine():
    fig, axs = plt.subplots(3,1, figsize=(6,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    TempC = 273.15
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Xticks = ['Basecase', r'S$_{30}$', r'S$_{60}$', r'S$_{90}$']
    
    ax1=plt.subplot(311)
    X = [RiparianTempB['temp-annual_avg'][-1], RiparianTempS30['temp-annual_avg'][-1],RiparianTempS60['temp-annual_avg'][-1],RiparianTempS90['temp-annual_avg'][-1]]
    X = [x - TempC for x in X]
    Y = [HillslopeTempB['temp-annual_avg'][-1], HillslopeTempS30['temp-annual_avg'][-1],HillslopeTempS60['temp-annual_avg'][-1],HillslopeTempS90['temp-annual_avg'][-1]]
    Y = [x - TempC for x in Y]
    ax1.plot(xpos,X,'r*-',label='Wetland')
    ax1.plot(xpos,Y,'k*-',label='Upland')
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')

    ax1.set_ylabel('Annual average [C]',fontsize='large', fontweight='normal')
    ax1.set_ylim((266.15 - TempC, 271.15- TempC))
    #ax1.set_title('Wetland',fontweight='normal')
    #ax1.text(3.2, 267- TempC, 'Annual average',fontsize=11, fontweight='normal')
    ax1.set_title('Soil temperatures at wetland and upland',fontsize=11, fontweight='normal')
    ax1.legend(loc='upper left', fontsize=10)#, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    plt.tight_layout()

    
    # --------------------------------- MAXIMUM ---------------------------
    ax1=plt.subplot(312)
    X = [RiparianTempB['temp-annual_max'][-1], RiparianTempS30['temp-annual_max'][-1],RiparianTempS60['temp-annual_max'][-1],RiparianTempS90['temp-annual_max'][-1]]
    X = [x - TempC for x in X]
    Y = [HillslopeTempB['temp-annual_max'][-1], HillslopeTempS30['temp-annual_max'][-1],HillslopeTempS60['temp-annual_max'][-1],HillslopeTempS90['temp-annual_max'][-1]]
    Y = [x - TempC for x in Y]
    ax1.plot(xpos,X,'r*-',label='Wetland')
    ax1.plot(xpos,Y,'k*-',label='Upland')
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    ax1.set_ylabel('Annual maximum [C]',fontsize='large', fontweight='normal')
    ax1.set_ylim((280.15- TempC, 281.15- TempC))
    #ax1.text(3.2, 275.6- TempC, 'Annual maximum',fontsize=11, fontweight='normal')
    
    ax1.legend(loc='upper left', fontsize=10)#, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    plt.tight_layout()
    
    # --------------------------------- MANIMUM ---------------------------
    ax1=plt.subplot(313)
    X = [RiparianTempB['temp-annual_min'][-1], RiparianTempS30['temp-annual_min'][-1],RiparianTempS60['temp-annual_min'][-1],RiparianTempS90['temp-annual_min'][-1]]
    X = [x - TempC for x in X]
    Y = [HillslopeTempB['temp-annual_min'][-1], HillslopeTempS30['temp-annual_min'][-1],HillslopeTempS60['temp-annual_min'][-1],HillslopeTempS90['temp-annual_min'][-1]]
    Y = [x - TempC for x in Y]
    ax1.plot(xpos,X,'r*-',label='Wetland')
    ax1.plot(xpos,Y,'k*-',label='Upland')
    ax1.set_xticks(xpos)
    ax1.set_xticklabels(Xticks,fontsize=11, fontweight='normal')
    
    ax1.set_xlabel('Simulation scenario',fontsize='large', fontweight='normal')
    ax1.set_ylabel('Annual minimum [C]',fontsize='large', fontweight='normal')
    
    ax1.set_ylim((255.15- TempC, 259.15- TempC))
    #ax1.text(3.2, 264.6- TempC, 'Annual minimum',fontsize=11, fontweight='normal')
    ax1.legend(loc='upper left', fontsize=10)#, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
    plt.tight_layout()
    
    plt.tight_layout()
    plt.savefig(outfile+'avg-max-min-temp-%scm-line.pdf'%depth_cm, bbox_inches='tight', dpi=100)
    plt.show()

    #riparian zone are more warmer due to sensible heat, support it that ALT (thawing) is more in the hillslope then if its not
    #warmer than either evaporative cooling or energy is lost in thawing, is it more thaw there then?

In [ ]:
#PlotAnnualAverage()
#PlotAnnualAvgMaxMin()
#PlotAnnualAvgMaxMinBAR()
PlotAnnualAvgMaxMinLine()

In [ ]:
def get_average_temp_bar():
    Xtemp = ['Annual Average', 'Annual Max', 'Annual Min']
    ypos = range(0,100,25)
    xpos = np.array([0, 4, 8, 12, 16])
    plt.bar(xpos-1.5,  Base_pc_area , align='edge', alpha=1.0, color='b', width=0.75,label='Basecase')
    plt.bar(xpos - 0.75,  EF_pc_area , align='edge', alpha=1., color='k', width=0.75,label='Early fall')
    plt.bar(xpos + 0.,  LF_pc_area , align='edge', alpha=1., color='g', width=0.75,label='Late fall')
    plt.bar(xpos+.75,  ELF_pc_area , align='edge', alpha=1., color='r', width=0.75,label='Early/late fall')
    plt.xticks(xpos, Xtemp,fontsize=10, fontweight='bold')
    plt.ylim(0,100)
    plt.yticks(ypos, fontsize=10, fontweight='bold')
    plt.ylabel('Soil temperature [K]',fontsize=10, fontweight='bold')
    plt.xlabel('Simulations scenarios',fontsize=10, fontweight='bold')
    #plt.title('Programming language usage')
    plt.legend()
    plt.tight_layout()
    plt.savefig(outfile+'avg-max-min-temp-50cm-bar.pdf', bbox_inches='tight', dpi=100)
    
    plt.show()

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/MaxSnow/'
fig, axs = plt.subplots(1,1, figsize=(6,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
#axs = axs.ravel()
Loc = 0
#HillslopeALT_B_loc10 = get_locationbased_ALT("B",location=Loc)
#HillslopeALT_S30_loc10 = get_locationbased_ALT("S30",location=Loc)
#HillslopeALT_S60_loc10 = get_locationbased_ALT("S60",location=Loc)
HillslopeALT_S90_loc10 = get_locationbased_ALT("S90",location=Loc)
print HillslopeALT_B_loc10['times-annual_avg'].shape

yr = 18
ax1=plt.subplot(111)
ax1.plot(HillslopeALT_B_loc10['times-annual_avg'][yr], HillslopeALT_B_loc10['temp-annual_avg'][yr],'k-',label='Basecase')
ax1.plot(HillslopeALT_S30_loc10['times-annual_avg'][yr], HillslopeALT_S30_loc10['temp-annual_avg'][yr],'g-',label=r'S$_{30}$')
ax1.plot(HillslopeALT_S60_loc10['times-annual_avg'][yr], HillslopeALT_S60_loc10['temp-annual_avg'][yr],'b-',label=r'S$_{60}$')
ax1.plot(HillslopeALT_S90_loc10['times-annual_avg'][yr], HillslopeALT_S90_loc10['temp-annual_avg'][yr],'r-',label=r'S$_{90}$')

print HillslopeALT_S90_loc10['times-annual_avg'][yr][0]%365
"""
yr = 18
ax2=plt.subplot(122)
ax2.plot(HillslopeALT_B_loc10['times-annual_avg'][yr], HillslopeALT_B_loc10['temp-annual_avg'][yr],'k-',label='Basecase')
ax2.plot(HillslopeALT_S30_loc10['times-annual_avg'][yr], HillslopeALT_S30_loc10['temp-annual_avg'][yr],'g-',label=r'S$_{30}$')
ax2.plot(HillslopeALT_S60_loc10['times-annual_avg'][yr], HillslopeALT_S60_loc10['temp-annual_avg'][yr],'b-',label=r'S$_{60}$')
ax2.plot(HillslopeALT_S90_loc10['times-annual_avg'][yr], HillslopeALT_S90_loc10['temp-annual_avg'][yr],'m-',label=r'S$_{90}$')
"""
ax1.set_title('Active layer thickness',fontsize='large', fontweight='normal')
ax1.set_xlabel('Time [Month of year]',fontsize='large', fontweight='normal')
ax1.set_ylabel('Depth [cm]',fontsize='large', fontweight='normal')
ax1.set_xlim(18.71,19.71)
#ax1.set_xticks(np.linspace(0,1,5))
#ax1.set_xticklabels(['Oct 19', 'Dec 31', 'March 16', 'May 26', 'July '],fontsize=11, fontweight='normal')
ax1.set_xticklabels(['x', 'Oct', 'Dec', 'March', 'May', 'July'],fontsize=11, fontweight='normal')
ax1.set_ylim((-61, 1.5))
ax1.legend(loc='upper left', fontsize=10)#, ncol=4, bbox_to_anchor=(1.0,-0.35, .1, .1))
  
plt.tight_layout()
#plt.savefig(outfile+'ALT-x%sm-1.pdf'%Loc, bbox_inches='tight', dpi=100)
plt.show()

3 30 [51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
 75 76 77 78 79 80]
[-40.61529 -34.09387 -30.04937 -26.43821 -23.21396 -20.33517 -17.76482
 -15.46986 -13.42079 -11.59127  -9.95776  -8.49928  -7.19706  -6.03436
  -4.99624  -4.06934  -3.24176  -2.50284  -1.8431   -1.25404  -0.72809
  -0.2585    0.16078   0.53514   0.86938   1.16782   1.43428   1.67689
   1.90257   2.11251   2.3078    2.48946   2.65845   2.81565   2.96189
   3.09792   3.22446   3.34217   3.45167   3.55353   3.64828   3.73643
   3.81842   3.89469   3.96565   4.03165   4.09304   4.15016   4.20329
   4.25271   4.29868   4.34145   4.38123   4.41824   4.45266   4.48469
   4.51448   4.54219   4.56796   4.59194   4.61425   4.635     4.655
   4.675     4.695     4.715     4.735     4.755     4.775     4.795
   4.815     4.835     4.855     4.875     4.895     4.915     4.935
   4.955     4.975     4.995     5.015  ] 4.34145 51
(19, 365)
18.710472279260777


In [ ]:
print 0.71*365, 0.8*365, .2*365, 0.4*365, 0.6*365
print 31 + 28 + 31 + 30 + 31 + 30 + 31 + 31 + 30 + 15#sept
print 31 + 28 + 31 + 30 + 31 # 146
print 31 + 28 + 31 + 30 + 31 + 30 + 31 #219

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
axs = axs.ravel()

HillslopeALT_B = dict()
for key in HillslopeALT_B_loc10.keys():
    if 'annual' in key:
        HillslopeALT_B[key] = np.concatenate((HillslopeALT_B_loc10[key]))
        HillslopeALT_EF[key] = np.concatenate((HillslopeALT_EF_loc10[key]))
        HillslopeALT_LF[key] = np.concatenate((HillslopeALT_LF_loc10[key]))
        HillslopeALT_EW[key] = np.concatenate((HillslopeALT_EW_loc10[key]))
        HillslopeALT_LW[key] = np.concatenate((HillslopeALT_LW_loc10[key]))
        HillslopeALT_ELF[key] = np.concatenate((HillslopeALT_ELF_loc10[key]))
"""        
plt.plot(HillslopeALT_B['times-annual_avg'], HillslopeALT_B['temp-annual_avg'],'k-',label='Base')
plt.plot(HillslopeALT_EF['times-annual_avg'], HillslopeALT_EF['temp-annual_avg'],'g-',label='Early fall')
plt.plot(HillslopeALT_LF['times-annual_avg'], HillslopeALT_LF['temp-annual_avg'],'b-',label='Late fall')
plt.plot(HillslopeALT_EW['times-annual_avg'], HillslopeALT_EW['temp-annual_avg'],'m-',label='Early winter')
plt.plot(HillslopeALT_LW['times-annual_avg'], HillslopeALT_LW['temp-annual_avg'],'c-',label='Late winter')
plt.plot(HillslopeALT_ELF['times-annual_avg'], HillslopeALT_ELF['temp-annual_avg'],'r-',label='Early/late fall')
"""
ax1 = plt.subplot(231)
ax1.plot(HillslopeALT_B['temp-annual_avg'],'k-',label='Base')
ax1.plot(HillslopeALT_EF['temp-annual_avg'],'g-',label='Early fall')
ax1.plot(HillslopeALT_LF['temp-annual_avg'],'b-',label='Late fall')
ax1.plot(HillslopeALT_EW['temp-annual_avg'],'m-',label='Early winter')
ax1.plot(HillslopeALT_LW['temp-annual_avg'],'c-',label='Late winter')
ax1.plot(HillslopeALT_ELF['temp-annual_avg'],'r-',label='Early/late fall')
#ax1.set_xlim((170,460))
ax1.set_xlim((170,545))

ax2 = plt.subplot(232)
ax2.plot(HillslopeALT_B['temp-annual_avg'],'k-',label='Base')
ax2.plot(HillslopeALT_EF['temp-annual_avg'],'g-',label='Early fall')
ax2.plot(HillslopeALT_LF['temp-annual_avg'],'b-',label='Late fall')
ax2.plot(HillslopeALT_EW['temp-annual_avg'],'m-',label='Early winter')
ax2.plot(HillslopeALT_LW['temp-annual_avg'],'c-',label='Late winter')
ax2.plot(HillslopeALT_ELF['temp-annual_avg'],'r-',label='Early/late fall')
ax2.set_xlim((530,835))

ax3 = plt.subplot(233)
ax3.plot(HillslopeALT_B['temp-annual_avg'],'k-',label='Base')
ax3.plot(HillslopeALT_EF['temp-annual_avg'],'g-',label='Early fall')
ax3.plot(HillslopeALT_LF['temp-annual_avg'],'b-',label='Late fall')
ax3.plot(HillslopeALT_EW['temp-annual_avg'],'m-',label='Early winter')
ax3.plot(HillslopeALT_LW['temp-annual_avg'],'c-',label='Late winter')
ax3.plot(HillslopeALT_ELF['temp-annual_avg'],'r-',label='Early/late fall')
ax3.set_xlim((880,1265))
#ax3.set_xlim((1255,1590))

ax4 = plt.subplot(234)
ax4.plot(HillslopeALT_B['temp-annual_avg'],'k-',label='Base')
ax4.plot(HillslopeALT_EF['temp-annual_avg'],'g-',label='Early fall')
ax4.plot(HillslopeALT_LF['temp-annual_avg'],'b-',label='Late fall')
ax4.plot(HillslopeALT_EW['temp-annual_avg'],'m-',label='Early winter')
ax4.plot(HillslopeALT_LW['temp-annual_avg'],'c-',label='Late winter')
ax4.plot(HillslopeALT_ELF['temp-annual_avg'],'r-',label='Early/late fall')
ax4.set_xlim((1255,1590))
#ax4.set_xlim((1627,1990))

ax5 = plt.subplot(235)
ax5.plot(HillslopeALT_B['temp-annual_avg'],'k-',label='Base')
ax5.plot(HillslopeALT_EF['temp-annual_avg'],'g-',label='Early fall')
ax5.plot(HillslopeALT_LF['temp-annual_avg'],'b-',label='Late fall')
ax5.plot(HillslopeALT_EW['temp-annual_avg'],'m-',label='Early winter')
ax5.plot(HillslopeALT_LW['temp-annual_avg'],'c-',label='Late winter')
ax5.plot(HillslopeALT_ELF['temp-annual_avg'],'r-',label='Early/late fall')
ax5.set_xlim((1627,1990))
plt.show()

In [ ]:
outfile='/Users/ajc/Documents/MyPapers/snowstorms/'
fig, axs = plt.subplots(1,1, figsize=(8,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
ax=plt.subplot(111)
ax.plot(HillslopeALT_B['times-annual_avg'], HillslopeALT_B['temp-annual_avg'],'k-',label='Base')
ax.plot(HillslopeALT_EF['times-annual_avg'], HillslopeALT_EF['temp-annual_avg'],'g-',label='Early fall')
ax.plot(HillslopeALT_LF['times-annual_avg'], HillslopeALT_LF['temp-annual_avg'],'b-',label='Late fall')
ax.plot(HillslopeALT_EW['times-annual_avg'], HillslopeALT_EW['temp-annual_avg'],'m-',label='Early winter')
ax.plot(HillslopeALT_LW['times-annual_avg'], HillslopeALT_LW['temp-annual_avg'],'c-',label='Late winter')
ax.plot(HillslopeALT_ELF['times-annual_avg'], HillslopeALT_ELF['temp-annual_avg'],'r-',label='Early/late fall')
ax.set_xlabel('Day of year',fontsize='large', fontweight='normal')
ax.set_ylabel('Thaw depth [cm]',fontsize='large', fontweight='normal')
ax.set_xlim((25., 31))
ax.set_ylim((-62, 1))
ax.set_xticklabels([2010,2011, 2012,2013,2014,2015,2016])
ax.legend(loc='upper left', ncol=1, bbox_to_anchor=(.0, 0.95),fontsize=8)
ax.set_title('Thaw depth at x = 0.0 m',fontweight='normal')
plt.tight_layout()
plt.savefig(outfile+'ALT-x0m.pdf', bbox_inches='tight', dpi=100)
plt.show()
plt.show()


In [ ]:
fig, axs = plt.subplots(1,1, figsize=(8,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)
plt.tight_layout()
print Data_EF['temperature'].shape
ax=plt.subplot(111)
mesh = Data_EF['temperature'][2,100,:,:]
ax.imshow(Data_EF['temperature'][2,100,:,:])

